# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

## Лабораторная работа 12

In [1]:
import dask.bag as db
import json
import re
import glob
import pandas as pd
from datetime import datetime

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов с помощью модуля `json`. Выведите на экран первые 5 элементов полученного `bag`.

In [2]:
files = glob.glob("data/reviews_full/*.json")
jsons = db.read_text(files).map(json.loads)
jsons.take(5)

({'user_id': 2056668,
  'recipe_id': 77845,
  'date': '2018-06-13',
  'review': "A terrific snack for the boys! They gobbled them up. Such a simple and tasty treat. Love that they are low calorie, but it's true, you can't just eat one, or two....they were so good! Thanks Annacia for another way to serve a easy snack. Made for ZWT3 ~V"},
 {'user_id': 1098259,
  'recipe_id': 253902,
  'date': '2020-04-01',
  'review': 'i have to say this was very easy i tend to have problems with my crust \nbut not with this recipe.  I&#039;m making pot pies tonight and this recipe \nwas perfect for it, thank you'},
 {'user_id': 452592,
  'recipe_id': 1775453,
  'date': '2009-07-12',
  'review': 'Not a fan.'},
 {'user_id': 1012958,
  'recipe_id': 892246,
  'date': '2016-12-20',
  'review': 'Perfect!  And smelled so good while simmering.  This made just the right amount for two 12" pizzas and tasted fabulous.  Easy, too.  Winner!!'},
 {'user_id': 2074896,
  'recipe_id': 1947864,
  'date': '2007-08-19',
  

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [3]:
def add_rating(data):
    item = json.loads(data[0])
    item['rating'] = re.findall(r'\d+', data[1])[-1]
    return item
rating_jsons = db.read_text(files, include_path=True).map(add_rating)
rating_jsons.take(5)

({'user_id': 2056668,
  'recipe_id': 77845,
  'date': '2018-06-13',
  'review': "A terrific snack for the boys! They gobbled them up. Such a simple and tasty treat. Love that they are low calorie, but it's true, you can't just eat one, or two....they were so good! Thanks Annacia for another way to serve a easy snack. Made for ZWT3 ~V",
  'rating': '1'},
 {'user_id': 1098259,
  'recipe_id': 253902,
  'date': '2020-04-01',
  'review': 'i have to say this was very easy i tend to have problems with my crust \nbut not with this recipe.  I&#039;m making pot pies tonight and this recipe \nwas perfect for it, thank you',
  'rating': '1'},
 {'user_id': 452592,
  'recipe_id': 1775453,
  'date': '2009-07-12',
  'review': 'Not a fan.',
  'rating': '1'},
 {'user_id': 1012958,
  'recipe_id': 892246,
  'date': '2016-12-20',
  'review': 'Perfect!  And smelled so good while simmering.  This made just the right amount for two 12" pizzas and tasted fabulous.  Easy, too.  Winner!!',
  'rating': '1'},
 {'u

3. Посчитайте количество отзывов в исходном датасете.

In [4]:
jsons.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [5]:
filtered_jsons = rating_jsons.filter(lambda obj: 2013 < datetime.strptime(obj['date'], '%Y-%m-%d').year < 2016)
filtered_jsons.take(5)

({'user_id': 774287,
  'recipe_id': 1299912,
  'date': '2015-09-27',
  'review': "WOW!  These are soooo good!  Best oven rib recipe I've tried yet.  I used Cattleman's original bbq sauce and threw in a little liquid smoke, also.  Thank you for sharing this awesome recipe!",
  'rating': '1'},
 {'user_id': 135872,
  'recipe_id': 487082,
  'date': '2015-11-12',
  'review': "I was nervous about trying this!  The last couple of recipes using ground turkey..weren't our favorite recipes..these are very very good!!  I will make this again! Instead of white bread, I used bread crumbs other than that, I followed as written. Thanks Cheryl!!",
  'rating': '1'},
 {'user_id': 416401,
  'recipe_id': 739313,
  'date': '2014-10-08',
  'review': 'I may never be as decadent as the Raj during the Empress of India days but I do love unusual sandwich fillings to spice up my really bland lunch periods. I didn&#039;t make tea sandwiches but I did make several lunchtime centerpieces with the fillings listed. E

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [6]:
def preprocessor(data):
    data['review'] = re.sub(r'[^a-zA-Z ]+', '', data['review'].lower().strip())
    return data
    
proccessed_jsons = filtered_jsons.map(preprocessor)
proccessed_jsons.take(5)

({'user_id': 774287,
  'recipe_id': 1299912,
  'date': '2015-09-27',
  'review': 'wow  these are soooo good  best oven rib recipe ive tried yet  i used cattlemans original bbq sauce and threw in a little liquid smoke also  thank you for sharing this awesome recipe',
  'rating': '1'},
 {'user_id': 135872,
  'recipe_id': 487082,
  'date': '2015-11-12',
  'review': 'i was nervous about trying this  the last couple of recipes using ground turkeywerent our favorite recipesthese are very very good  i will make this again instead of white bread i used bread crumbs other than that i followed as written thanks cheryl',
  'rating': '1'},
 {'user_id': 416401,
  'recipe_id': 739313,
  'date': '2014-10-08',
  'review': 'i may never be as decadent as the raj during the empress of india days but i do love unusual sandwich fillings to spice up my really bland lunch periods i didnt make tea sandwiches but i did make several lunchtime centerpieces with the fillings listed everything turned out super but

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [7]:
proccessed_jsons.count().compute()

735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка.

In [12]:
jsons_df = proccessed_jsons.to_dataframe()
jsons_df['rating'].value_counts().compute()

5    528231
4    119413
0     42472
3     26532
2      9380
1      9246
Name: rating, dtype: int64

8. Найдите среднее значение `rating` в наборе, полученном в задании 5.

In [16]:
jsons_df['rating'].mean().compute()

ValueError: `mean` not supported with object series

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [20]:
def binop(t, x):
    return max((t, x), key=lambda x: x[1])

proccessed_jsons = proccessed_jsons.map(lambda item: (item['rating'], len(item['review']))).foldby(lambda item: item[0], binop)

In [22]:
proccessed_jsons.compute()

[('1', ('1', 2868)),
 ('0', ('0', 6548)),
 ('3', ('3', 3174)),
 ('2', ('2', 2834)),
 ('5', ('5', 5343)),
 ('4', ('4', 6548))]